## 2. Gram-Schmidt 

### 2.1. 개요
  
  - 직교화(orthogonalization)는 벡터공간에서 서로 직교하는 정규 기저1(orthonormal basis)를 찾는 과정으로서 수치적 선형대수가 활용되는 대부분의 분야에서 빈번하게 사용됨.
  
  - Gram-Schmidt 과정은 직교화의 방법 중 하나로 다음과 같은 알고리즘으로 진행.
  
    <img src = "images/image12.png" width="70%" height="70%">

    <img src = "images/image13.png" width="70%" height="70%">

  - 이를 전개하여 표현하면 다음곽 같음.

  $$\bf{u} _1 = \bf{v} _1$$
  
  $$\bf{u} _2 = \bf{v} _2 - \frac{\langle \bf{v _2}, \bf{u}_{1} \rangle}{\left\| \bf{u}_{1} \right\| ^{2}} \bf{u}_{1} $$

  $$\bf{u} _3 = \bf{v} _3 - \frac{\langle \bf{v _3}, \bf{u}_{2} \rangle}{\left\| \bf{u}_{2} \right\| ^{2}} \bf{u}_{2}-\frac{\langle \bf{v _3}, \bf{u}_{1} \rangle}{\left\| \bf{u}_{1} \right\| ^{2}} \bf{u}_{1}$$
    
  $$ \vdots $$
  
  $$ \bf{u} _n 
    = \bf{v} _n - \frac{\langle \bf{v _n}, \bf{u}_{n-1} \rangle}{\left\| \bf{u}_{n-1} \right\| ^{2}} \bf{u}_{n-1}
    - \frac{\langle \bf{v _n}, \bf{u}_{n-2} \rangle}{\left\| \bf{u}_{n-2} \right\| ^{2}} \bf{u}_{n-2}
    - \cdots 
    - \frac{\langle \bf{v _n}, \bf{u}_{1} \rangle}{\left\| \bf{u}_{1} \right\| ^{2}} \bf{u}_{1}$$

  - 2차원 벡터에 대한 Gram-Schmidt 과정
  
  <img src = "images/image14.png">


### 2.2. QR 분해

  - 다음과 같이 $\bf{A}$를 분해

  $$ \bf{A} = \bf{QR} $$

  - 여기서 $\bf{Q}$의 열벡터는 $\bf{A}$의 열벡터가 span하는 공간 $S$의 직교 기저 벡터이며, $\bf{R}$는 Upper triangular matrix임
  
  - $\bf{A}$ 의 열벡터 ${ \bf{a}_i}$ 들은 Gram-Schmidt 과정을 거쳐 얻어진 정규 직교 벡터 ${ (\bf{u}_1, \bf{u}_2, \cdots, \bf{u}_n)}$ 에 대해 다음을 만족
  
  $$ \bf{a} _i
  = \langle \bf{a} _i, \bf{u}_{1} \rangle \bf{u}_{1} + 
  \langle \bf{a} _i, \bf{u}_{2} \rangle \bf{u}_{2} + \cdots +
  \langle \bf{a} _i, \bf{u}_{n} \rangle \bf{u}_{n} $$

In [4]:
import numpy as np

# 10개의 20차원 벡터
N = 10
D = 20
v = np.random.rand(D, N)

np.save("v", v)

In [ ]:
# QR 분해 : Q의 열벡터가 정규기저벡터
import numpy as np

v = np.load("v.npy")
print(v)
Q, R = np.linalg.qr(v)
print(Q)
print(R)
print(np.linalg.norm(Q[0]))

1. 순차코드
   
  - 열벡터를 행벡터로 전환
  - 메모리 접근이 효율적임

In [ ]:
import numpy as np

V = np.transpose(v).copy()
print(V)
for j in range (N) :
    for i in range (j) :
        coef = -np.dot(V[i], V[j])
        V[j] = V[j] + coef * V[i]
    coef = np.linalg.norm(V[j])
    V[j] = V[j] / coef

print(V)

2. 병렬코드

  - 의존성 분석
    - $u_i$들은 순차적으로 계산됨 : $u_i$를 위해 $u_{i-1}, \cdots, u_{1}$이 필요
    - $u_i, v_i$의 각 $i$ 성분 계산은 독립적임 : 이를 나누어 계산하며 벡터 분할과 동일
  
  <img src = "images/image03.png">

In [ ]:
%%writefile gs.py
import numpy as np
from mpi4py import MPI
from tools import para_range

N = 3
D = 9

comm = MPI.COMM_WORLD

size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0 : 
    v = np.load("v.npy")
    V = np.transpose(v).copy()
else :
    V = np.empty((N, D), dtype = np.float64)

ista, iend = para_range(D, size, rank)

chunk = iend - ista + 1

# Scatterv에 필요한 cnts와 disp 계산
cnts = #FIX ME
disp = []
for i in range (size) :
    disp.append #FIX ME

V_chunk = np.empty([N, chunk], dtype = np.float64)

# V의 분할. 모든 V[i]에 대한 분할 필요
for i in range (N) :
    comm.Scatterv #FIX ME
    
# 분할 계산후 reduction
for j in range (N) :
    for i in range (j) :
        coef = -np.dot #FIX ME
        coef_all = #FIX ME
        V_chunk[j] = V_chunk[j] + coef_all * V_chunk[i]
    coef = np.dot(V_chunk[j], V_chunk[j])
    coef_all = #FIX ME
    V_chunk[j] = V_chunk[j] / np.sqrt(coef_all)

print(V_chunk)

for i in range (N) :
    comm.Gatherv( V_chunk[i], (V[i],cnts), root = 0)

if rank == 0 :
    print(V)
    


In [ ]:
! mpirun -np 4 python gs.py